In [11]:
### Part (a): PyPlate Implementation
# Importing necessary packages and objects from PyPlate
from pyplate import Plate, Substance, Container, Recipe
import random

# Set the random seed for reproducibility
random.seed(42)

# Define real molecular weights and density for Pd catalyst and ligands
Pd_catalyst = Substance.solid(name='Pd(OAc)2', mol_weight=224.50)
Pd_catalyst_container = Container("Pd_catalyst", initial_contents=[(Pd_catalyst, "10 mL")])

# Ligands
xphos = Substance.solid(name='XPhos', mol_weight=345.41)
sphos = Substance.solid(name='SPhos', mol_weight=295.35)
dppf = Substance.solid(name='dppf', mol_weight=554.63)
ligandsS = [xphos, sphos, dppf]

# Ligand Containers
xphos_container = Container("XPhos", initial_contents=[(xphos, "10 mL")])
sphos_container = Container("SPhos", initial_contents=[(sphos, "10 mL")])
dppf_container = Container("dppf", initial_contents=[(dppf, "10 mL")])
ligandsC = [xphos_container, sphos_container, dppf_container]

# Solvents
toluene = Substance.liquid(name="toluene", mol_weight=92.141, density=0.8623)
glyme = Substance.liquid(name="glyme", mol_weight=90.12, density=0.868)
TBME = Substance.liquid(name="TBME", mol_weight=88.15, density=0.74)
dichloroethane = Substance.liquid(name="dichloroethane", mol_weight=98.95, density=1.253)
solventsS = [toluene, glyme, TBME, dichloroethane]

# Solvent Containers
toluene_container = Container("toluene", initial_contents=[(toluene, "10 mL")])
glyme_container = Container("glyme", initial_contents=[(glyme, "10 mL")])
TBME_container = Container("TBME", initial_contents=[(TBME, "10 mL")])
dichloroethane_container = Container("dichloroethane", initial_contents=[(dichloroethane, "10 mL")])
solventsC = [toluene_container, glyme_container, TBME_container, dichloroethane_container]


# Generate molecular weights for Ai and Bi using a random number generator
Ai_substances = [Substance.solid(name=f"A{i}", mol_weight=round(random.uniform(10, 200),2)) for i in range(12)]
Bi_substances = [Substance.solid(name=f"B{i}", mol_weight=round(random.uniform(10, 200),2)) for i in range(12)]

Ai_containers = [Container(f"A{i}_C",  initial_contents=[(Ai_substances[i], "10 ML")])
                 for i in range(12)]
Bi_containers = [Container(f"B{i}_C",  initial_contents=[(Bi_substances[i], "10 ML")])
                 for i in range(12)]

In [26]:
# Initialize a 96-well plate with max volumne of 500 uL per well.
plate = Plate('96_well_plate', max_volume_per_well='500 uL', rows=8, columns=12)

In [27]:
# Create a new recipe and add the components to the recipe
recipe = Recipe().uses(plate,Pd_catalyst_container)

for i in range(len(Ai_containers)):
    recipe.uses(Ai_containers[i])
    recipe.uses(Bi_containers[i])
    
for i in range(len(ligandsC)):
    recipe.uses(ligandsC[i])
    
for i in range(len(solventsC)):
    recipe.uses(solventsC[i])

In [28]:
def calculate_volume(molar_mass_g_mol, concentration_mmol):
    # Convert molar mass from g/mol to mg/mmol (mmol/mL) and calculate volume
    concentration_mg_mmol = molar_mass_g_mol / 1000  # Convert g/mol to mg/mmol
    volume_ml = concentration_mmol / concentration_mg_mmol
    return str(round(volume_ml,3)) + " uL" 

def calculate_remaining_volumne(Ai, Bi, catalyst, ligand, total):
    remain = float(total.split(" ")[0]) - (float(Ai.split(" ")[0]) + float(Bi.split(" ")[0]) +  float(catalyst.split(" ")[0]) + float(ligand.split(" ")[0]))
    return str(remain) + " uL" 

In [29]:
for row in range(plate.n_rows):
    for col in range(plate.n_columns):
        ligand_index = col // 4
        solvent_index = col // 3
        AiC = Ai_containers[col]
        BiC = Bi_containers[col]
    
        # Transfer 0.1 mmol of Ai
        Ai = Ai_substances[col]
        Ai_volume = calculate_volume(Ai.mol_weight, 0.1)
        
        # Transfer 1.1 equivalents of Bi
        Bi = Bi_substances[col]
        Bi_volume = calculate_volume(Bi.mol_weight, 0.1 * 1.1)
        
        # Transfer 10 mol% equivalents of Pd(OAc)2
        catalyst_volume = calculate_volume(Pd_catalyst.mol_weight, 0.1 * 0.10)
        
        ligandC = ligandsC[ligand_index]
        ligandS = ligandsS[ligand_index]
        # Transfer 10 mol% equivalents of ligand 
        ligand_volume = calculate_volume(ligandS.mol_weight, 0.1 * 0.15)
        
        solventC = solventsC[solvent_index]
        
        recipe.transfer(AiC, plate[row+1, col+1], Ai_volume)
        recipe.transfer(BiC, plate[row+1, col+1], Bi_volume)        
        recipe.transfer(Pd_catalyst_container, plate[row+1, col+1], catalyst_volume)
        recipe.transfer(ligandC, plate[row+1, col+1], ligand_volume)
        
        # Fill the remaining volumne of each well with the solvent
        remaining_volume = calculate_remaining_volumne(Ai_volume, Bi_volume, catalyst_volume, ligand_volume, "200 uL")
        recipe.transfer(solventC, plate[row+1, col+1], remaining_volume)

results = recipe.bake()
plate = results[plate.name]
# recipe.visualize(what=plate, mode='final', unit='uL', timeframe=0)

In [36]:
recipe.visualize(what=plate, mode='final', unit='uL', timeframe='all')

,1,2,3,4,5,6,7,8,9,10,11,12
A,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
B,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
C,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
D,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
E,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
F,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
G,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
H,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0


In [37]:
### Part a documentaion

Here's a brief documentation for the provided code:

### Import Statements:
        The code begins by importing necessary packages and objects from PyPlate, which is a library for managing chemical reactions and substances.
        Plate, Substance, Container, and Recipe are imported from PyPlate.
        The random module is imported for generating random numbers.

### Random Seed:
        The random seed is set to 42 using random.seed(42) for reproducibility of random number generation.

### Defining Substances and Containers:
        Pd_catalyst: A solid substance representing Pd(OAc)2 with a molecular weight of 224.50. It is stored in a container named "Pd_catalyst" with an initial volume of 10 mL.
        Ligands (xphos, sphos, dppf): Solid substances representing different ligands with specific molecular weights. These ligands are stored in separate containers (xphos_container, sphos_container, dppf_container) each with an initial volume of 10 mL.
        Solvents (toluene, glyme, TBME, dichloroethane): Liquid substances representing different solvents with molecular weights and densities. Each solvent is stored in its respective container with an initial volume of 10 mL.
        Ai_substances and Bi_substances: Lists of solid substances (A0 to A11 and B0 to B11 respectively) generated using random molecular weights between 10 and 200. Each substance is stored in its corresponding container with an initial volume of 10 mL.

    Summary:
        The code sets up substances representing catalysts, ligands, solvents, and randomly generated substances, storing them in containers with specific initial volumes.
        This setup is likely for simulating chemical reactions or experiments where these substances and containers are needed.



In [6]:
### Part (b): PyPlate API Modification Proposal


To introduce the concept of tags and relative quantities like "1.1 * A" in PyPlate, we would need to make the following changes:

1. **Substance Class**: Modify the `Substance` class to include a property for tags, which would be a list of strings.

2. **Transfer Method**: Enhance the transfer method in the `Recipe` class to accept expressions involving tags, parse them, and calculate the relative quantities.

3. **Validation**: Implement a validation method to ensure physical reasonableness, such as checking that the final volume does not exceed the well capacity or that the sum of equivalents does not exceed stoichiometric constraints.

4. **Documentation**: Update docstrings in the `Substance` and `Recipe` classes to reflect the new tagging feature and usage.

Here's an example of how the docstrings might be updated to explain the new API behavior:

### Substance Class

#### Updated Attributes:
- `tags`: list of str
  List of string labels associated with the substance to be used in specifying relative quantities.


### Recipe Class

#### Updated Methods:

##### transfer(source, destination, quantity, ...)
Transfers a quantity of a substance from a source container to a destination container. The quantity can now be specified using tags that represent relative amounts of other substances involved in the recipe.

Example:
```python
# Associate the substance with a tag
A1.tag('A')

# Use the tag in a transfer to specify a relative quantity
recipe.transfer(source=B1, destination=well, quantity="1.1 * A")
```

This would transfer an amount of B1 equivalent to 1.1 times the amount associated with the tag 'A'.

#### Validation Method:

##### validate_quantities(...)
Ensures that all quantities specified in the recipe are physically reasonable. Checks include verifying that well volumes are not exceeded and that relative amounts do not result in stoichiometrically impossible scenarios.

### Ensuring Physical Reasonableness

To ensure that quantities are physically reasonable, implement checks such as:

- Total volume in a well should not exceed its maximum capacity.
- The sum of equivalents based on tags must be stoichiometrically plausible.
- For liquid transfers, ensure that the density of the substance is taken into account to avoid overfilling.
- If a limiting reagent is specified, ensure that other reagents' amounts are coherent with the stoichiometry.

Unit tests could be written to validate the correctness of these checks under various scenarios, ensuring robustness and reliability of the recipe execution.